In [ ]:
from google.colab import drive
import os
import pandas as pd
import shutil

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
outfits= os.listdir("/content/drive/MyDrive/295B_Project/colorful_fashion_dataset_for_object_detection/JPEGImages/")

In [ ]:
!unzip -u '/content/drive/MyDrive/295B_Project/product_images_nov20/product_images_nov20.zip'

In [ ]:
products = os.listdir('/content/content/product_images_nov20/')

In [ ]:
products

In [ ]:
import torch
from torchvision import models, transforms
import torch.nn as nn
from PIL import Image

In [ ]:
weights = models.ResNet50_Weights.IMAGENET1K_V1
model = models.resnet50(weights=weights)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 70.5MB/s]


In [ ]:
!pip install fastai
!pip install gdown
!pip install tldextract
# !pip install Pillow==6.2.0
!pip uninstall annoy
!pip install annoy==1.17.1

In [ ]:
products_dict = {}
for name in products:
  base_name = name.split('_')[0]+'.jpg'  # This will now give only the numeric part before the first underscore.
  print(base_name)  # This will give you 'img_548'
  if base_name in outfits:
        if base_name not in products_dict:
            products_dict[base_name] = []
        products_dict[base_name].append(name.rsplit('_', 1)[-1].split('.')[0])

In [ ]:
# products_dict['100073.jpg']

['shirt', 'shoe', 'hat', 'skirt']

In [ ]:
df = pd.DataFrame(products_dict.items(), columns=['outfit_names', 'products'])

In [ ]:
df.to_csv('df_nov20.csv',index=False)

In [ ]:
import torch
from torchvision import models, transforms
import torch.nn as nn
from annoy import AnnoyIndex
scene_image_folder = '/content/drive/MyDrive/295B_Project/colorful_fashion_dataset_for_object_detection/JPEGImages/'
scene_images = os.listdir(scene_image_folder)
weights = models.ResNet50_Weights.IMAGENET1K_V1
model = models.resnet50(weights=weights)

model.fc = nn.Identity()
# print(model)

model.eval()

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
    ])

annoy_index = AnnoyIndex(2048, 'angular')

for i in range(len(scene_images)):
  image = Image.open(os.path.join(scene_image_folder, scene_images[i]))
  input_tensor = transform(image).unsqueeze(0)

  if input_tensor.size()[1] == 3:
    output_tensor = model(input_tensor)
    annoy_index.add_item(i, output_tensor[0])

    if i % 100 == 0:
      print(f"Processed {i} images")
    # print(f'{image} predicted as {weights.meta["categories"][torch.argmax(output_tensor)]}')

annoy_index.build(n_trees= 20,n_jobs = -1)
annoy_index.save('scene_index.ann')

In [ ]:
scene_image_folder = '/content/drive/MyDrive/295B_Project/colorful_fashion_dataset_for_object_detection/JPEGImages'
scene_images = os.listdir(scene_image_folder)

In [ ]:
from PIL import ImageDraw,Image

# scene_image_grid = Image.new('RGB', (300,300))
outfit_dict ={}
for i in range(len(scene_images)):
  image = Image.open(os.path.join(scene_image_folder, scene_images[i]))
  input_tensor = transform(image).unsqueeze(0)

  if input_tensor.size()[1] == 3:
    output_tensor = model(input_tensor)
    nns = annoy_index.get_nns_by_vector(output_tensor[0], 50)
    outfit_list = []

    for j in range(len(nns)):
      outfit_list.append(scene_images[nns[j]])

    outfit_dict[scene_images[i]] = outfit_list

In [ ]:
outfit_dict['1.jpg']

In [ ]:
import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/295B_Project/df/df_unique_final.csv')

In [ ]:
products_list = os.listdir('/content/content/product_images_nov20')

In [ ]:
products = pd.read_csv('/content/df_nov20.csv')

In [ ]:
# accessories = ['hat','shoe','bag']
labels = ['shirt','jacket']

In [ ]:
products_dict = products.set_index('outfit_names')['products'].to_dict()

In [ ]:
labels_set = set(labels)

missing_labels_dict = {}

for outfit, products in products_dict.items():
    missing_accessories = [item for item in labels if item not in products]
    missing_labels_dict[outfit] = missing_accessories

In [ ]:
updated_dict = {}

for key,similar_outfits in outfit_dict.items():
#   print(a)
  product_rec = []
  missing_accessories = missing_labels_dict.get(key, [])
  # print(missing_accessories)
  for similar_outfit in similar_outfits:
    potential_rec = [similar_outfit[:-4] + '_' + accessory + '.jpg' for accessory in missing_accessories]
    for name in potential_rec:
      if name in products_list:
        product_rec.append(name)
  updated_dict[key] = product_rec

In [ ]:
updated_dict['100034.jpg']

['165390_jacket.jpg',
 '13525_jacket.jpg',
 '128940_jacket.jpg',
 '4163_jacket.jpg',
 '193647_jacket.jpg',
 '34725_jacket.jpg',
 '96551_jacket.jpg',
 '147043_jacket.jpg',
 '25837_jacket.jpg',
 '31643_jacket.jpg',
 '181777_jacket.jpg',
 '174137_jacket.jpg',
 '172089_jacket.jpg']

In [ ]:
from PIL import Image
import os

labels = ['shirt','jacket']

for key, value in updated_dict.items():
    for label in labels:
        base_image_name = key[:-4]  # Remove the file extension from the key to get the base image name
        base_dir = f'/content/colorful_final_recommendations/{label}_recommendations_final/{base_image_name}'
        os.makedirs(base_dir, exist_ok=True)

        # Path for the base image
        base_outfit_path = '/content/drive/MyDrive/295B_Project/colorful_fashion_dataset_for_object_detection/JPEGImages/' + key
        base_image = Image.open(base_outfit_path)

        # Save the base image itself
        base_image.save(os.path.join(base_dir, f'{base_image_name}.png'))

        # Save each recommendation image specific to the current label
        recommendation_counter = 1
        for product_name in value:
            # print(product_name.rsplit('_', 1)[-1].split('.')[0])
            if product_name.rsplit('_', 1)[-1].split('.')[0] == label:
                product_image_path = '/content/content/product_images_nov20/' + product_name
                product_image = Image.open(product_image_path)
                recommendation_filename = f'{base_image_name}_{product_name[:-4]}_{recommendation_counter}.jpg'
                product_image.save(os.path.join(base_dir, recommendation_filename))
                recommendation_counter += 1  # Increment the counter after each save

In [ ]:
import os
import shutil

main_dir = '/content/colorful_final_recommendations/jacket_recommendations_final'

# Iterate over all directories within the main directory
for label in os.listdir(main_dir):
    label_dir = os.path.join(main_dir, label)

    # Make sure it's a directory
    if os.path.isdir(label_dir):
        # List all items in the label directory
        contents = os.listdir(label_dir)

        # Check if there's only one file in the directory
        if len(contents) == 1:
            # Construct the full path to the file
            file_path = os.path.join(label_dir, contents[0])

            # Make sure the single content is a file, not a subdirectory
            if os.path.isfile(file_path):
                # Delete the directory
                shutil.rmtree(label_dir)
                print(f"Deleted {label_dir} because it contained only one file.")

In [ ]:
import os
import shutil

main_dir = '/content/colorful_final_recommendations/shirt_recommendations_final'

# Iterate over all directories within the main directory
for label in os.listdir(main_dir):
    label_dir = os.path.join(main_dir, label)

    # Make sure it's a directory
    if os.path.isdir(label_dir):
        # List all items in the label directory
        contents = os.listdir(label_dir)

        # Check if there's only one file in the directory
        if len(contents) == 1:
            # Construct the full path to the file
            file_path = os.path.join(label_dir, contents[0])

            # Make sure the single content is a file, not a subdirectory
            if os.path.isfile(file_path):
                # Delete the directory
                shutil.rmtree(label_dir)
                print(f"Deleted {label_dir} because it contained only one file.")

In [ ]:
bags = os.listdir('/content/colorful_final_recommendations/jacket_recommendations_final')

In [ ]:
len(bags)

1856

In [ ]:
!zip -r /content/colorful_final_recommendations.zip /content/colorful_final_recommendations

Streaming output truncated to the last 5000 lines.
  adding: content/colorful_final_recommendations/shirt_recommendations_final/121266/121266_41727_shirt_17.jpg (deflated 6%)
  adding: content/colorful_final_recommendations/shirt_recommendations_final/121266/121266_8732_shirt_27.jpg (deflated 4%)
  adding: content/colorful_final_recommendations/shirt_recommendations_final/121266/121266_117375_shirt_1.jpg (deflated 4%)
  adding: content/colorful_final_recommendations/shirt_recommendations_final/121266/121266_68535_shirt_31.jpg (deflated 4%)
  adding: content/colorful_final_recommendations/shirt_recommendations_final/121266/121266_45173_shirt_8.jpg (deflated 2%)
  adding: content/colorful_final_recommendations/shirt_recommendations_final/121266/121266_178572_shirt_10.jpg (deflated 6%)
  adding: content/colorful_final_recommendations/shirt_recommendations_final/121266/121266_38182_shirt_20.jpg (deflated 2%)
  adding: content/colorful_final_recommendations/shirt_recommendations_final/12126

In [ ]:
missing_accessories_dict = {key: values for key, values in missing_accessories_dict.items() if values}

In [ ]:
updated_dict = {}

for key,similar_outfits in outfit_dict.items():
  product_rec = []
  missing_accessories = missing_accessories_dict.get(key, [])
#   print(missing_accessories)
  for similar_outfit in similar_outfits:
    potential_rec = [similar_outfit[:-4] + '_' + accessory + '.jpg' for accessory in missing_accessories]
    for name in potential_rec:
      if name in products_list:
        product_rec.append(name)
  updated_dict[key] = product_rec

In [ ]:
from PIL import Image

# Assuming transform and unsqueeze are defined earlier in the code
# and new_dict_pants is your dictionary of outfits with recommendations

for key, value in updated_dict.items():
    # Create a directory for the base image
    base_image_name = key[:-4]  # Remove the file extension from the key to get the base image name
    base_dir = f'/content/shoe_recommendations_final_1/{base_image_name}'
    os.makedirs(base_dir, exist_ok=True)

    # Path for the base image
    base_outfit_path = '/content/content/labeled_outfits_nov20/' + key
    base_image = Image.open(base_outfit_path).resize((256, 256))

    # Save the base image itself
    base_image.save(os.path.join(base_dir, f'{base_image_name}.jpg'))

    # Save each recommendation image
    recommendation_counter = 1
    for product_name in value:
        if(product_name.split('_', 1)[-1].split('.')[0]== 'shoe'):
            # print(product_name.split('_', 1)[-1].split('.')[0])
            product_image_path = '/content/content/product_images_nov20/' + product_name
            product_image = Image.open(product_image_path).resize((256, 256))
            recommendation_filename = f'{base_image_name}_{product_name.split("_")[0]}_recommendation_{recommendation_counter}.jpg'
            product_image.save(os.path.join(base_dir, recommendation_filename))
            recommendation_counter += 1  # Increment the counter after each save


In [ ]:
shutil.copytree("/content/shoe_recommendations_final_1","/content/drive/MyDrive/shoe_recommendations_final_1")

'/content/drive/MyDrive/shoe_recommendations_final_1'

In [ ]:
from PIL import Image
import os

# Assuming transform and unsqueeze are defined earlier in the code
# and new_dict_pants is your dictionary of outfits with recommendations

for key, value in updated_dict.items():
    # Create a directory for the base image
    base_image_name = key[:-4]  # Remove the file extension from the key to get the base image name
    base_dir = f'/content/bag_recommendations_final_1/{base_image_name}'
    os.makedirs(base_dir, exist_ok=True)

    # Path for the base image
    base_outfit_path = '/content/content/labeled_outfits_nov20/' + key
    base_image = Image.open(base_outfit_path).resize((256, 256))

    # Save the base image itself
    base_image.save(os.path.join(base_dir, f'{base_image_name}.png'))

    # Save each recommendation image
    recommendation_counter = 1
    for product_name in value:
        if(product_name.split('_', 1)[-1].split('.')[0]== 'bag'):
            # print(product_name.split('_', 1)[-1].split('.')[0])
            product_image_path = '/content/content/product_images_nov20/' + product_name
            product_image = Image.open(product_image_path).resize((256, 256))
            recommendation_filename = f'{base_image_name}_{product_name.split("_")[0]}_recommendation_{recommendation_counter}.png'
            product_image.save(os.path.join(base_dir, recommendation_filename))
            recommendation_counter += 1  # Increment the counter after each save


In [ ]:
shutil.copytree("/content/bag_recommendations_final_1","/content/drive/MyDrive/bag_recommendations_final_1")

'/content/drive/MyDrive/bag_recommendations_final_1'

In [ ]:
# Replace 'your_folder_path' with the path of the folder you want to zip
!zip -r /content/shoe_recommendations_final_1.zip /content/shoe_recommendations_final_1

# Copy the zip file to your Google Drive
shutil.copy('/content/shoe_recommendations_final_1.zip', '/content/drive/MyDrive/295B_Project/shoe_recommendations_final_1.zip')


In [ ]:
# Replace 'your_folder_path' with the path of the folder you want to zip
!zip -r /content/bag_recommendations_final_1.zip /content/bag_recommendations_final_1

# Copy the zip file to your Google Drive
shutil.copy('/content/bag_recommendations_final_1.zip', '/content/drive/MyDrive/295B_Project/bag_recommendations_final_1.zip')


In [ ]:
os.listdir("/content/bag_recommendations_final_1/192514")

['192514_22264_recommendation_6.png',
 '192514_60265_recommendation_2.png',
 '192514_150805_recommendation_1.png',
 '192514_70282_recommendation_3.png',
 '192514_186280_recommendation_13.png',
 '192514_118758_recommendation_9.png',
 '192514_64292_recommendation_8.png',
 '192514_10084_recommendation_7.png',
 '192514_88531_recommendation_18.png',
 '192514_14439_recommendation_17.png',
 '192514_7660_recommendation_5.png',
 '192514_25517_recommendation_10.png',
 '192514_99040_recommendation_16.png',
 '192514_27992_recommendation_12.png',
 '192514_187750_recommendation_15.png',
 '192514_59715_recommendation_11.png',
 '192514_100922_recommendation_14.png',
 '192514_172826_recommendation_4.png',
 '192514.png']